In [1]:
%matplotlib widget

In [2]:
import xtrack as xt

### Lattice defined in a programmatic way (python)

This is like in MAD-X. The lattice and the strengths are defined as pyhon scripts (which can include logic, comments, etc.).
It works well also on large lattices, see for example [lhc_seq.py](https://raw.githubusercontent.com/xsuite/xtrack/refs/heads/main/examples/lhc_native_python/lhc_seq.py) (we plan to generate these files directly from the layout database and make them available in the acc-models repository).

In [3]:
%%capture
env = xt.Environment()        # Create an environment

env.call('pimm_seq.py')       # Load lattice
env.call('pimm_strengths.py') # Load optics

In [4]:
tw = env.ring.twiss4d()                 # Twiss 
tw.rows['qfa.*'].cols['s betx bety dx'] # Get optics at selected locations

TwissTable: 8 rows, 5 cols
name              s          betx          bety            dx
qfa.0        2.2125       9.53142       4.10532   1.80855e-05
qfa.1        8.1525       12.3793       4.86072       2.47195
qfa.2       28.1175        12.995       4.44476       2.73962
qfa.3       34.0575       9.24421       4.54585   1.77093e-05
qfa.4       36.8325       9.53142       4.10532   1.80813e-05
qfa.5       42.7725       12.3793       4.86072       2.47195
qfa.6       62.7375        12.995       4.44476       2.73962
qfa.7       68.6775       9.24421       4.54585   1.77056e-05

### Lattice stored as json

In [5]:
env = xt.Environment.from_json('pimm.json') # Load lattice
env.vars.load_json('pimm_strengths.json')   # Load optics

Loading line from dict:   0%|          | 0/145 [00:00<?, ?it/s]

Done loading line from dict.           


In [6]:
tw = env.ring.twiss4d()                 # Twiss 
tw.rows['qfa.*'].cols['s betx bety dx'] # Get optics at selected locations

TwissTable: 8 rows, 5 cols
name              s          betx          bety            dx
qfa.0        2.2125       9.53142       4.10532   1.80855e-05
qfa.1        8.1525       12.3793       4.86072       2.47195
qfa.2       28.1175        12.995       4.44476       2.73962
qfa.3       34.0575       9.24421       4.54585   1.77093e-05
qfa.4       36.8325       9.53142       4.10532   1.80813e-05
qfa.5       42.7725       12.3793       4.86072       2.47195
qfa.6       62.7375        12.995       4.44476       2.73962
qfa.7       68.6775       9.24421       4.54585   1.77056e-05

### Load existing MAD-X sequence sequence optics files (without MADX/cpymad)

MAD-X files containing only data (e.g. sequence and optics files from repositories) are compatible with Xsuite and can be read directly. This feature will be maintained also after MAD-X support discontinuation.

In [62]:
env = xt.load_madx_lattice('./PIMM.seq')                    # Load lattice
env.vars.load_madx('pimm_strengths.madx')                   # Load optics
env.ring.particle_ref = xt.Particles(kinetic_energy0=200e6) # Define ref. particle

In [63]:
tw = env.ring.twiss4d()                 # Twiss 
tw.rows['qfa.*'].cols['s betx bety dx'] # Get optics at selected locations

TwissTable: 8 rows, 5 cols
name              s          betx          bety            dx
qfa.1        2.2125       8.54912       4.65644     -0.240056
qfa.2        8.1525       14.7618       10.8065       2.37992
qfa.3       28.9425       15.5887       9.76334       2.62585
qfa.4       34.8825       8.41509       5.92293     -0.288397
qfa.5       39.8325       8.54912       4.65644     -0.240056
qfa.6       45.7725       14.7618       10.8065       2.37992
qfa.7       66.5625       15.5887       9.76334       2.62585
qfa.8       72.5025       8.41509       5.92293     -0.288397

### Use MAD-X itself through cpymad to read the file and import to Xsuite
‼️ This supports any MAD-X script. It will be discontinued when MAD-X support will end (~2029)


In [66]:
from cpymad.madx import Madx
madx = Madx()
madx.call('PIMM.seq')
madx.call('pimm_strengths.madx')
madx.beam()
madx.use('ring')

# Load cpymad sequence object as Xsuite line
ring = xt.Line.from_madx_sequence(madx.sequence.ring, deferred_expressions=True)
ring.particle_ref = xt.Particles(kinetic_energy0=200e6)


  ++++++++++++++++++++++++++++++++++++++++++++
  +     MAD-X 5.09.02  (64 bit, Darwin)      +
  + Support: mad@cern.ch, http://cern.ch/mad +
  + Release   date: 2024.04.18               +
  + Execution date: 2025.02.19 14:18:58      +
  ++++++++++++++++++++++++++++++++++++++++++++
! Based on CERN-PS-99-010-DI, TERA Foundation



mb:  sbend, l=1.661, angle=0.3926990817, e1=0.19634954085, e2=0.19634954085;

qfa: quadrupole, l=2*0.175, k1:= kqfa;

qd:  quadrupole, l=2*0.175, k1:= kqd;

qfb: quadrupole, l=2*0.175, k1:= kqfb;



sd1: sextupole, l=0.2, k2:=ksd;

sf1: sextupole, l=0.2, k2:=ksf;

sd2: sextupole, l=0.2, k2:=ksd;

sf2: sextupole, l=0.2, k2:=ksf;

se1: sextupole, l=0.2, k2:=kse;



cav: rfcavity, volt:=vrf, freq:=frf, lag=0;



ring:sequence,refer=centre,l=75.24;

              extr_septum: marker, at=0.0;

              qfa.1: qfa, at=2.3875;

              mb, at=3.8125;

              qd.1: qd, at=5.2925;

              mb, at=7.0475;

              qfa.2: qfa, at=8.3275;

  

Converting sequence "ring":   0%|          | 0/96 [00:00<?, ?it/s]

In [67]:
tw = ring.twiss4d()                     # Twiss 
tw.rows['qfa.*'].cols['s betx bety dx'] # Get optics at selected locations

TwissTable: 8 rows, 5 cols
name              s          betx          bety            dx
qfa.1        2.2125       8.54912       4.65644     -0.240056
qfa.2        8.1525       14.7618       10.8065       2.37992
qfa.3       28.9425       15.5887       9.76334       2.62585
qfa.4       34.8825       8.41509       5.92293     -0.288397
qfa.5       39.8325       8.54912       4.65644     -0.240056
qfa.6       45.7725       14.7618       10.8065       2.37992
qfa.7       66.5625       15.5887       9.76334       2.62585
qfa.8       72.5025       8.41509       5.92293     -0.288397